In [5]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DevelopmentConfig, ProductionConfig
from app.models import MetaData


if os.environ.get('APP_MODE') == 'production':
    config = ProductionConfig()
else:
    config = DevelopmentConfig()

engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
Session = sessionmaker(bind=engine)
session = Session()

meta_data = session.query(MetaData).first()
meta_data

<MetaData 1>

In [10]:
meta_data.LastUpdated

'May2024'

In [1]:
import pandas as pd
import sqlite3
import os
pd.set_option('display.max_columns', None)

from app import db, create_app
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# DB_LOC = os.environ.get('DB_LOC')
# UPLOAD_LOC= os.environ.get('UPLOAD_LOC')

db_path_main = 'instance/database.db'
db_path_backup = 'data/backups/instance/database.db'

In [3]:
with sqlite3.connect(db_path_main) as conn:
    # df = pd.read_sql_query("SELECT LastUpdated FROM meta_data", conn)
    main_df = pd.read_sql_query("""
        SELECT IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,
               NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,
               PrimaryLocation,SecondaryLocation,District,Neighborhood,
               NeighborhoodNum,Latitude,Longitude,Supplemented,
               SupplementDate,VictimNum,FirearmUsed,IncidentNature
        FROM crime_data
        """, conn)
main_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,19007979,2021-01-13,00:01:00,FAIL TO REGISTER AS A SEX OFFENDER PURSUANT TO...,90Z,Unspecified,All Other Offenses,NaN,F,1915 OLIVE ST,None,4.0,Downtown West,36,38.632110,-90.207689,Yes,2023-09-19 10:18:30.0930000,NaN,No,Offenses - All Other - Criminal
1,19035349,2021-10-26,10:30:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT,240,Property,Motor Vehicle Theft,7.0,F,LINDENWOOD,MACKLIND,2.0,Southampton,7,38.594193,-90.292975,Yes,2021-11-01 10:38:08.0000000,NaN,No,Motor Vehicle Theft - Criminal
2,19035349,2021-10-26,10:30:00,STEALING,26A,Property,False Pretense/Swindle/Confidence Game,NaN,M,LINDENWOOD,MACKLIND,2.0,Southampton,7,38.594193,-90.292975,Yes,2021-11-01 10:38:08.0000000,NaN,No,Motor Vehicle Theft - Criminal
3,19058248,2023-10-10,18:40:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT,240,Property,Motor Vehicle Theft,7.0,F,N 9TH,LUCAS,1.0,Downtown,35,38.639401,-90.189785,Yes,2023-10-11 11:48:58.6600000,NaN,No,Motor Vehicle Theft - Criminal
4,19061654,2021-08-24,01:15:00,STEALING/LARCENY/THEFT,23G,Property,Theft From Motor Vehicle Parts/Accessories,6.0,C,S KINGSHIGHWAY / CHIPPEWA,None,2.0,Southampton,7,NaN,NaN,Yes,2021-08-24 03:12:00.0000000,NaN,No,Larceny - Criminal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201066,24017858,2024-04-19,15:00:00,STEALING,23F,Property,Theft From Motor Vehicle,6.0,M,3101 LAFAYETTE AVE,None,3,The Gate District,31,38.616441,-90.230398,No,None,NaN,No,Larceny - Criminal
201067,24017859,2024-04-30,16:00:00,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,90Z,Unspecified,All Other Offenses,NaN,None,4000 LINCOLN AVE,None,5,The Greater Ville,56,38.656898,-90.230820,No,None,NaN,No,Offenses - All Other - Criminal
201068,24017859,2024-04-30,16:00:00,PROPERTY DAMAGE 1ST DEGREE,90Z,Unspecified,All Other Offenses,NaN,F,4000 LINCOLN AVE,None,5,The Greater Ville,56,38.656898,-90.230820,No,None,NaN,No,Offenses - All Other - Criminal
201069,24017947,2024-04-25,00:01:00,STEALING - $750 OR MORE,23H,Property,All Other Larceny,6.0,F,1300 AUBERT AVE,None,5,Fountain Park,53,38.660261,-90.260475,No,None,NaN,No,Larceny - Criminal


In [5]:
main_df = main_df.sort_values(['IncidentDate', 'IncidentNum']).reset_index(drop=True)
main_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,21000002,2021-01-01,00:02:00,MISCELLANEOUS WEAPON VIOLATION,520,Society,Weapons Law Violations,NaN,None,1700 CHESTNUT ST,None,4.0,Downtown West,36,38.629573,-90.204995,No,None,NaN,Yes,Weapon Law Violation - Criminal
1,21000003,2021-01-01,00:09:00,PROPERTY DAMAGE,290,Property,Destruction/Damage/Vandalism of Property,NaN,None,4961 LACLEDE AVE,210,5.0,Central West End,38,38.641221,-90.264226,Yes,None,NaN,No,Destruction of Property - Criminal
2,21000003,2021-01-01,00:09:00,MISCELLANEOUS WEAPON VIOLATION,520,Society,Weapons Law Violations,NaN,None,4961 LACLEDE AVE,210,5.0,Central West End,38,38.641221,-90.264226,Yes,None,NaN,No,Destruction of Property - Criminal
3,21000007,2021-01-01,00:10:00,PROPERTY DAMAGE - 2ND DEGREE,290,Property,Destruction/Damage/Vandalism of Property,NaN,None,6963 PERNOD AVE,None,2.0,Lindenwood Park,9,38.598725,-90.310722,No,None,NaN,No,Destruction of Property - Criminal
4,21000008,2021-01-01,00:20:00,PROPERTY DAMAGE - 2ND DEGREE,290,Property,Destruction/Damage/Vandalism of Property,NaN,None,3500 KINGSLAND CT,None,1.0,Dutchtown,16,38.579494,-90.243210,No,None,NaN,No,Offenses - All Other - Criminal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201066,24017716,2024-04-30,17:00:00,STEALING - $750 OR MORE,23F,Property,Theft From Motor Vehicle,6.0,F,300 N 8TH ST,None,4,Downtown,35,38.628753,-90.192243,No,None,NaN,No,Larceny - Criminal
201067,24017721,2024-04-30,20:00:00,STEALING - FIREARM/EXPLOSIVE WEAPON/AMMONIUM N...,23F,Property,Theft From Motor Vehicle,6.0,F,4715 BEACON AVE,None,6,Walnut Park East,72,38.692107,-90.256702,No,None,NaN,No,Larceny - Criminal
201068,24017815,2024-04-30,18:00:00,STEALING UNDER $150 (PETTY LARCENY-MOTOR VEH P...,23G,Property,Theft From Motor Vehicle Parts/Accessories,6.0,None,3217 GEYER AVE,None,3,Compton Heights,26,38.614722,-90.233410,No,None,NaN,No,Larceny - Criminal
201069,24017859,2024-04-30,16:00:00,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,90Z,Unspecified,All Other Offenses,NaN,None,4000 LINCOLN AVE,None,5,The Greater Ville,56,38.656898,-90.230820,No,None,NaN,No,Offenses - All Other - Criminal


In [4]:
with sqlite3.connect(db_path_backup) as conn:
    # df = pd.read_sql_query("SELECT LastUpdated FROM meta_data", conn)
    backup_df = pd.read_sql_query("""
        SELECT IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,
               NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,
               PrimaryLocation,SecondaryLocation,District,Neighborhood,
               NeighborhoodNum,Latitude,Longitude,Supplemented,
               SupplementDate,VictimNum,FirearmUsed,IncidentNature
        FROM crime_data
        """, conn)
backup_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,19007979,2021-01-13,00:01:00,FAIL TO REGISTER AS A SEX OFFENDER PURSUANT TO...,90Z,Unspecified,All Other Offenses,NaN,F,1915 OLIVE ST,None,4.0,Downtown West,36,38.632110,-90.207689,Yes,2023-09-19 10:18:30.0930000,NaN,No,Offenses - All Other - Criminal
1,19035349,2021-10-26,10:30:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT,240,Property,Motor Vehicle Theft,7.0,F,LINDENWOOD,MACKLIND,2.0,Southampton,7,38.594193,-90.292975,Yes,2021-11-01 10:38:08.0000000,NaN,No,Motor Vehicle Theft - Criminal
2,19035349,2021-10-26,10:30:00,STEALING,26A,Property,False Pretense/Swindle/Confidence Game,NaN,M,LINDENWOOD,MACKLIND,2.0,Southampton,7,38.594193,-90.292975,Yes,2021-11-01 10:38:08.0000000,NaN,No,Motor Vehicle Theft - Criminal
3,19058248,2023-10-10,18:40:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT,240,Property,Motor Vehicle Theft,7.0,F,N 9TH,LUCAS,1.0,Downtown,35,38.639401,-90.189785,Yes,2023-10-11 11:48:58.6600000,NaN,No,Motor Vehicle Theft - Criminal
4,19061654,2021-08-24,01:15:00,STEALING/LARCENY/THEFT,23G,Property,Theft From Motor Vehicle Parts/Accessories,6.0,C,S KINGSHIGHWAY / CHIPPEWA,None,2.0,Southampton,7,NaN,NaN,Yes,2021-08-24 03:12:00.0000000,NaN,No,Larceny - Criminal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193286,24017858,2024-04-19,15:00:00,STEALING,23F,Property,Theft From Motor Vehicle,6.0,M,3101 LAFAYETTE AVE,None,3,The Gate District,31,38.616441,-90.230398,No,None,NaN,No,Larceny - Criminal
193287,24017859,2024-04-30,16:00:00,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,90Z,Unspecified,All Other Offenses,NaN,None,4000 LINCOLN AVE,None,5,The Greater Ville,56,38.656898,-90.230820,No,None,NaN,No,Offenses - All Other - Criminal
193288,24017859,2024-04-30,16:00:00,PROPERTY DAMAGE 1ST DEGREE,90Z,Unspecified,All Other Offenses,NaN,F,4000 LINCOLN AVE,None,5,The Greater Ville,56,38.656898,-90.230820,No,None,NaN,No,Offenses - All Other - Criminal
193289,24017947,2024-04-25,00:01:00,STEALING - $750 OR MORE,23H,Property,All Other Larceny,6.0,F,1300 AUBERT AVE,None,5,Fountain Park,53,38.660261,-90.260475,No,None,NaN,No,Larceny - Criminal


In [6]:
backup_df = backup_df.sort_values(['IncidentDate', 'IncidentNum']).reset_index(drop=True)
backup_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,21000002,2021-01-01,00:02:00,MISCELLANEOUS WEAPON VIOLATION,520,Society,Weapons Law Violations,NaN,None,1700 CHESTNUT ST,None,4.0,Downtown West,36,38.629573,-90.204995,No,None,NaN,Yes,Weapon Law Violation - Criminal
1,21000003,2021-01-01,00:09:00,PROPERTY DAMAGE,290,Property,Destruction/Damage/Vandalism of Property,NaN,None,4961 LACLEDE AVE,210,5.0,Central West End,38,38.641221,-90.264226,Yes,None,NaN,No,Destruction of Property - Criminal
2,21000003,2021-01-01,00:09:00,MISCELLANEOUS WEAPON VIOLATION,520,Society,Weapons Law Violations,NaN,None,4961 LACLEDE AVE,210,5.0,Central West End,38,38.641221,-90.264226,Yes,None,NaN,No,Destruction of Property - Criminal
3,21000007,2021-01-01,00:10:00,PROPERTY DAMAGE - 2ND DEGREE,290,Property,Destruction/Damage/Vandalism of Property,NaN,None,6963 PERNOD AVE,None,2.0,Lindenwood Park,9,38.598725,-90.310722,No,None,NaN,No,Destruction of Property - Criminal
4,21000008,2021-01-01,00:20:00,PROPERTY DAMAGE - 2ND DEGREE,290,Property,Destruction/Damage/Vandalism of Property,NaN,None,3500 KINGSLAND CT,None,1.0,Dutchtown,16,38.579494,-90.243210,No,None,NaN,No,Offenses - All Other - Criminal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193286,24017716,2024-04-30,17:00:00,STEALING - $750 OR MORE,23F,Property,Theft From Motor Vehicle,6.0,F,300 N 8TH ST,None,4,Downtown,35,38.628753,-90.192243,No,None,NaN,No,Larceny - Criminal
193287,24017721,2024-04-30,20:00:00,STEALING - FIREARM/EXPLOSIVE WEAPON/AMMONIUM N...,23F,Property,Theft From Motor Vehicle,6.0,F,4715 BEACON AVE,None,6,Walnut Park East,72,38.692107,-90.256702,No,None,NaN,No,Larceny - Criminal
193288,24017815,2024-04-30,18:00:00,STEALING UNDER $150 (PETTY LARCENY-MOTOR VEH P...,23G,Property,Theft From Motor Vehicle Parts/Accessories,6.0,None,3217 GEYER AVE,None,3,Compton Heights,26,38.614722,-90.233410,No,None,NaN,No,Larceny - Criminal
193289,24017859,2024-04-30,16:00:00,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,90Z,Unspecified,All Other Offenses,NaN,None,4000 LINCOLN AVE,None,5,The Greater Ville,56,38.656898,-90.230820,No,None,NaN,No,Offenses - All Other - Criminal


In [8]:
with sqlite3.connect(db_path_main) as conn:
    # df = pd.read_sql_query("SELECT LastUpdated FROM meta_data", conn)
    main_unf_df = pd.read_sql_query("""
        SELECT IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,
               NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,
               PrimaryLocation,SecondaryLocation,District,Neighborhood,
               NeighborhoodNum,Latitude,Longitude,Supplemented,
               SupplementDate,VictimNum,FirearmUsed,IncidentNature
        FROM unfounded_data
        """, conn)

In [9]:
main_unf_df = main_unf_df.sort_values(['IncidentDate', 'IncidentNum']).reset_index(drop=True)
main_unf_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,22000139,2021-01-01,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2022-01-18 10:28:43.9330000,None,None,None
1,21000277,2021-01-02,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,21000620,2021-01-03,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,21000416,2021-01-04,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,21000428,2021-01-04,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2021-01-27 09:18:40.2970000,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,24006331,2024-02-13,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2024-02-14,None,None,None
1232,24006855,2024-02-16,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2024-02-19,None,None,None
1233,24006912,2024-02-18,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1234,24006992,2024-02-18,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [10]:
with sqlite3.connect(db_path_backup) as conn:
    # df = pd.read_sql_query("SELECT LastUpdated FROM meta_data", conn)
    backup_unf_df = pd.read_sql_query("""
        SELECT IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,
               NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,
               PrimaryLocation,SecondaryLocation,District,Neighborhood,
               NeighborhoodNum,Latitude,Longitude,Supplemented,
               SupplementDate,VictimNum,FirearmUsed,IncidentNature
        FROM unfounded_data
        """, conn)

In [11]:
backup_unf_df = backup_unf_df.sort_values(['IncidentDate', 'IncidentNum']).reset_index(drop=True)
backup_unf_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,22000139,2021-01-01,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2022-01-18 10:28:43.9330000,None,None,None
1,21000277,2021-01-02,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,21000620,2021-01-03,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,21000416,2021-01-04,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,21000428,2021-01-04,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2021-01-27 09:18:40.2970000,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,24006331,2024-02-13,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2024-02-14,None,None,None
1210,24006855,2024-02-16,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2024-02-19,None,None,None
1211,24006912,2024-02-18,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1212,24006992,2024-02-18,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
main_

In [14]:
df1 = main_df
df2 = backup_df

# Perform an outer merge
merged_df = pd.merge(df1, df2, how='outer', indicator=True)

# Filter out the rows that are common in both dataframes
diff_df = merged_df[merged_df['_merge'] != 'both']

# Drop the indicator column
# diff_df = diff_df.drop(columns=['_merge'])

diff_df = diff_df.sort_values(['IncidentDate', 'IncidentNum']).reset_index(drop=True)
diff_df

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature,_merge
0,21000514,2021-01-01,00:46:00,MISCELLANEOUS WEAPON VIOLATION,520,Society,Weapons Law Violations,NaN,None,5110 MAFFITT AVE,None,5.0,Kingsway West,52,38.670699,-90.258630,Yes,None,NaN,Yes,Weapon Law Violation - Criminal,left_only
1,21001652,2021-01-11,15:00:00,BURGLARY - 2ND DEGREE,220,Property,Burglary/Breaking and Entering,5.0,None,4875 PENROSE ST,None,6.0,Penrose,69,38.680388,-90.244110,Yes,2024-02-05 14:10:14.4400000,NaN,No,Burglary - Criminal,left_only
2,21001878,2021-01-13,18:00:00,MURDER 1ST DEGREE,09A,Person,Murder and Nonnegligent Manslaughter,1.0,None,3520 NATURAL BRIDGE AVE,None,4.0,JeffVanderLou,59,38.660631,-90.216271,Yes,2022-12-02 19:22:27.9130000,214807.0,Yes,Homicide - Criminal,left_only
3,21003324,2021-01-23,01:43:00,ARMED CRIMINAL ACTION,90Z,Unspecified,All Other Offenses,NaN,F,3218 N. NEWSTEAD,None,6.0,The Greater Ville,56,38.666972,-90.238393,Yes,2024-01-22 12:46:29.1800000,NaN,No,Homicide - Criminal,left_only
4,21003324,2021-01-23,01:43:00,ARMED CRIMINAL ACTION,90Z,Unspecified,All Other Offenses,NaN,F,3218 N. NEWSTEAD,None,6.0,The Greater Ville,56,38.666972,-90.238393,Yes,2024-02-26 18:05:08.3800000,NaN,No,Homicide - Criminal,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5941,24012955,2024-03-31,20:30:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT,240,Property,Motor Vehicle Theft,7.0,F,5319 ALFRED AVE,None,1,Bevo Mill,5,38.580882,-90.270660,No,None,NaN,No,Motor Vehicle Theft - Criminal,left_only
5942,24012971,2024-03-31,02:33:00,DISCHARGING FIREARM WITHIN CITY (SHOTSPOTTER),520,Society,Weapons Law Violations,NaN,None,1107 TYLER ST,None,4,Old North St Louis,63,38.645492,-90.191000,Yes,4/3/2024 12:05:19 AM,NaN,Yes,Weapon Law Violation - Criminal,left_only
5943,24012982,2024-03-31,20:30:00,DISCHARGING FIREARM WITHIN CITY (SHOTSPOTTER),520,Society,Weapons Law Violations,NaN,None,5597 FLOY AVE,None,6,Walnut Park West,76,38.710110,-90.256300,Yes,4/2/2024 6:05:28 PM,NaN,Yes,Weapon Law Violation - Criminal,left_only
5944,24013244,2024-03-31,05:30:00,DOMESTIC ASSAULT - 2ND DEGREE,13B,Person,Simple Assault,NaN,F,1376 GRANVILLE PL,None,5,Hamilton Heights,78,38.665513,-90.275419,Yes,4/3/2024 11:11:00 AM,389720.0,No,Simple Assault - Criminal,left_only


In [15]:
diff_df['_merge'].unique()

['left_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [17]:
backup_df[backup_df['IncidentNum'] == 24012955]

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
189345,24012955,2024-03-31,20:30:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT,240,Property,Motor Vehicle Theft,7.0,F,5319 ALFRED AVE,None,1,Bevo Mill,5,38.580882,-90.27066,Yes,4/9/2024 5:27:25 PM,NaN,No,Motor Vehicle Theft - Criminal


In [18]:
main_df['IncidentNum'].value_counts()

IncidentNum
23057057    56
24001812    44
23018790    34
23051429    32
22019108    28
            ..
22023125     1
22022279     1
22020819     1
22019599     1
22039236     1
Name: count, Length: 139399, dtype: int64

In [19]:
main_df[main_df['IncidentNum'] == 23057057]

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
179944,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375330.0,Yes,Aggravated Assault - Shooting - Criminal
179945,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375331.0,Yes,Aggravated Assault - Shooting - Criminal
179946,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375332.0,Yes,Aggravated Assault - Shooting - Criminal
179947,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375333.0,Yes,Aggravated Assault - Shooting - Criminal
179948,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375334.0,Yes,Aggravated Assault - Shooting - Criminal
179949,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375335.0,Yes,Aggravated Assault - Shooting - Criminal
179950,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375336.0,Yes,Aggravated Assault - Shooting - Criminal
179951,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375337.0,Yes,Aggravated Assault - Shooting - Criminal
179952,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375338.0,Yes,Aggravated Assault - Shooting - Criminal
179953,23057057,2023-12-15,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,F,6165 LUCILLE AVE,None,6.0,Walnut Park West,76,38.706611,-90.25893,Yes,2024-02-15 18:05:42.3530000,375339.0,Yes,Aggravated Assault - Shooting - Criminal


In [21]:
# Perform an outer merge on column 'A'
merged_df = pd.merge(df1, df2, on='IncidentNum', how='outer', indicator=True)

# Filter out the rows that are common in both dataframes
diff_df = merged_df[merged_df['_merge'] != 'both']

diff_df

,IncidentNum,IncidentDate_x,TimeOccurred_x,SLMPDOffense_x,NIBRSCode_x,NIBRSCat_x,NIBRSOffenseType_x,SRS_UCR_x,CrimeGrade_x,PrimaryLocation_x,SecondaryLocation_x,District_x,Neighborhood_x,NeighborhoodNum_x,Latitude_x,Longitude_x,Supplemented_x,SupplementDate_x,VictimNum_x,FirearmUsed_x,IncidentNature_x,IncidentDate_y,TimeOccurred_y,SLMPDOffense_y,NIBRSCode_y,NIBRSCat_y,NIBRSOffenseType_y,SRS_UCR_y,CrimeGrade_y,PrimaryLocation_y,SecondaryLocation_y,District_y,Neighborhood_y,NeighborhoodNum_y,Latitude_y,Longitude_y,Supplemented_y,SupplementDate_y,VictimNum_y,FirearmUsed_y,IncidentNature_y,_merge
254398,23010523,2023-02-17,09:30:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT &...,240,Property,Motor Vehicle Theft,7.0,F,4295 KOSSUTH AVE,1E,6.0,OFallon,68,38.672108,-90.23058,Yes,2024-02-29,NaN,No,Motor Vehicle Theft - Criminal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [24]:
backup_unf_df[backup_unf_df['IncidentNum'] == 23010523]

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,SRS_UCR,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
845,23010523,2023-02-17,None,UNFOUNDED INCIDENT,None,None,None,None,None,None,None,None,None,None,None,None,None,2024-02-24,None,None,None
